In [8]:
import pandas as pd
import numpy as np
from scipy.stats import norm

In [9]:
class SwaptionPriceCalculator:
    def __init__(self, data, notional, option_maturity, swap_tenor, volatility, risk_free_rate):
        self.df = pd.DataFrame(data)
        self.notional = notional
        self.option_maturity = option_maturity
        self.volatility = volatility
        self.swap_tenor = swap_tenor
        self.risk_free_rate = risk_free_rate
        


Calculating forward rates, simple forward rates and discounting factors

Discount factor calculation for a given maturity

In [10]:
def calculate_strike_rate(self):
    maturity_index = int(self.swap_tenor * 2) -1
    strike_rate = 0
    for i in range(maturity_index):
        strike_rate += np.exp(-self.df['Maturity'][i] * self.df['Continuous_rate'][i])
    return strike_rate

In [11]:
def calculate_annuity(self):
    
    # Assuming semi-annual payments
    
    maturity_index = int(self.option_maturity *2) - 1 
    annuity_factor = 0
    for i in range(maturity_index, len(self.df['Maturity'])):
        annuity_factor += np.exp(-self.df['Maturity'][i] * self.df['Continuous_rate'][i])
    return annuity_factor

Performing analytical valuation of the swaps - Black 76 model
This will return us option price

In [12]:
def analytical_valuation(self):
    
    forward_swap_rate = self.calculate_stike_rate()
    
    d1 = (np.log(forward_swap_rate / self.calculate_strike_rate) + 0.5 * self.volatility ** 2 * self.option_maturity) / (self.volatility * np.sqrt(self.option_maturity))
    d2 = d1 - self.volatility * np.sqrt(self.option_maturity)
    N_d1 = norm.cdf(d1)
    N_d2 = norm.cdf(d2)
    annuity_factor = self.notional * self.swap_tenor * self.calculate_annuity()
    
    if self.option_type == 'receiver_swaption':
        option_price = (forward_swap_rate * N_d1 - self.calculate_strike_rate() * N_d2) * annuity_factor
    else:
        option_price = (self.calculate_stike_rate * (1-N_d2) - forward_swap_rate * (1 - N_d1)) * annuity_factor
    
    return option_price
          
          

In [13]:
def monte_carlo_valuation(self, F0, nsteps, nsims):
    
    dt = self.option_maturity / nsteps
    F = np.zeros((nsims, nsteps + 1))
    F[:, 0] = F0
    for i in range(nsteps):
        phi = np.random.randn(nsims)
        F[:, i + 1] = F[:, i] * np.exp(phi * self.volatility * np.sqrt(dt) - 0.5 * self.volatility ** 2 * dt)
        
    payoff = np.maximum(F[:, -1] - F0, 0)
    discounted_payoff = np.exp(-self.risk_free_rate * self.option_maturity) * payoff
    option_price = self.notional * 0.5 * self.calculate_annuity() * discounted_payoff.mean()
    
    return option_price
     

Testing our valuation model

In [14]:
data = {
    'Maturity': [0.5, 1.0, 1.5, 2.0, 2.5, 3.0],
    'Continuous_rate': [0.0500, 0.0600, 0.0675, 0.0725,0.0750, 0.0760],
}
notional = 1000000
option_maturity = 1
swap_tenor = 2
volatility = 0.2
risk_free_rate = 0.05

swaption_calculator = SwaptionPriceCalculator(data, notional, option_maturity, swap_tenor, volatility, risk_free_rate)
analytical_price = swaption_calculator.analytical_valuation()
print('Analytical Valuation Price : ', analytical_price)

AttributeError: 'SwaptionPriceCalculator' object has no attribute 'analytical_valuation'

In [ ]:
monte_carlo_price = swaption_calculator.monte_carlo_valuation(0.06, 100, 10000)
print('Monte Carlo Valustion Price : ', monte_carlo_price)

In [18]:
import numpy as np
from scipy.stats import norm

class SwaptionPriceCalculator:
    def __init__(self, data, notional, option_maturity, swap_tenor, volatility, risk_free_rate, option_type):
        self.data = data
        self.notional = notional
        self.option_maturity = option_maturity
        self.swap_tenor = swap_tenor
        self.volatility = volatility
        self.risk_free_rate = risk_free_rate
        self.option_type = option_type

    def calculate_annuity(self):
        maturity_index = int(self.option_maturity * 2) - 1  # Assuming semi-annual payments
        annuity_factor = 0
        for i in range(maturity_index, len(self.data['Maturity'])):
            annuity_factor += np.exp(-self.data['Maturity'][i] * self.data['Continuous_rate'][i])
        return annuity_factor

    def calculate_strike_rate(self):
        maturity_index = int(self.swap_tenor * 2) - 1  # Assuming semi-annual payments
        strike_rate = 0
        for i in range(maturity_index):
            strike_rate += np.exp(-self.data['Maturity'][i] * self.data['Continuous_rate'][i])
        return strike_rate

    def analytical_valuation(self):
        forward_swap_rate = self.calculate_strike_rate()
        d1 = (np.log(forward_swap_rate / self.calculate_strike_rate()) + 0.5 * self.volatility ** 2 * self.option_maturity) / (self.volatility * np.sqrt(self.option_maturity))
        d2 = d1 - self.volatility * np.sqrt(self.option_maturity)
        N_d1 = norm.cdf(d1)
        N_d2 = norm.cdf(d2)
        annuity_factor = self.notional * self.swap_tenor * self.calculate_annuity()
        
        if self.option_type == 'receiver_swaption':
            option_price = (forward_swap_rate * N_d1 - self.calculate_strike_rate() * N_d2) * annuity_factor
        else:
            option_price = (self.calculate_strike_rate() * (1 - N_d2) - forward_swap_rate * (1 - N_d1)) * annuity_factor
        
        return option_price

    def monte_carlo_valuation(self, F0, nsteps, nsims):
        dt = self.option_maturity / nsteps
        F = np.zeros((nsims, nsteps + 1))
        F[:, 0] = F0
        for i in range(nsteps):
            phi = np.random.randn(nsims)
            F[:, i + 1] = F[:, i] * np.exp(phi * self.volatility * np.sqrt(dt) - 0.5 * self.volatility ** 2 * dt)
        
        payoff = np.maximum(F[:, -1] - F0, 0)
        discounted_payoff = np.exp(-self.risk_free_rate * self.option_maturity) * payoff
        option_price = self.notional * 0.5 * self.calculate_annuity() * discounted_payoff.mean()
        
        return option_price

# Example usage:
data = {
    'Maturity': [0.5, 1.0, 1.5, 2.0, 2.5, 3.0],
    'Continuous_rate': [0.0500, 0.0600, 0.0675, 0.0725,0.0750, 0.0760],
}
notional = 1000000
option_maturity = 1
swap_tenor = 2
volatility = 0.2
risk_free_rate = 0.05
option_type = 'receiver_swaption'

swaption_calculator = SwaptionPriceCalculator(data, notional, option_maturity, swap_tenor, volatility, risk_free_rate, option_type)
analytical_price = swaption_calculator.analytical_valuation()
monte_carlo_price = swaption_calculator.monte_carlo_valuation(0.06, 100, 10000)

print("Analytical Valuation Price:", analytical_price)
print("Monte Carlo Valuation Price:", monte_carlo_price)


Analytical Valuation Price: 1948364.0585207187
Monte Carlo Valuation Price: 9880.933106353872
